In [ ]:
#Import necessary libraries and moules
import os
import pandas as pd
from dataclasses import dataclass
from pathlib import Path
from sklearn.model_selection import train_test_split
from src.constants import *
from src.utils.common import read_yaml, create_directories
from src.logging import logger


In [ ]:
#Get the current working directory
%pwd

In [ ]:
#Change directory to parent directory
os.chdir("../")
%pwd

In [ ]:
#Configuration class for Data Transformation component
@dataclass(frozen = True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [ ]:
#Configuration Manager class to handle configurations
class ConfigurationManager:

    #Initialization method to read config, params, and schema files
    def __init__(
        self,
        config_filePath = CONFIG_FILE_PATH,
        params_filePath = PARAMS_FILE_PATH,
        schema_filePath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filePath)
        self.params = read_yaml(params_filePath)
        self.schema = read_yaml(schema_filePath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir = config.root_dir,
            data_path = config.data_path
        )
        return data_transformation_config
                            

In [ ]:
#Data Transformation class to handle data splitting (Data Transformation component)
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    #Method to split data into training and testing sets
    def train_test_splitting(self):
        data = pd.read_csv(self.config.data_path)
        
        #Splitting the dataset into training and testing sets
        train,test = train_test_split(data, test_size = 0.2, random_state = 42)

        #Saving the training and testing sets to CSV files
        train.to_csv(os.path.join(self.config.root_dir, "train.csv"), index = False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index = False)

        logger.info("Splitted dataset into training and test sets")
        logger.info(train.shape)
        logger.info(test.shape)

In [ ]:
#Pipline to execute the Data Transformation component
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config = data_transformation_config)
    data_transformation.train_test_splitting()
except Exception as e:
    logger.exception(e)
    raise e